In [1]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
playlists_uris = []

In [5]:
def playlists(link):
    playlist_URI = link.split("/")[-1]
    try:
        playlist_URI = playlist_URI.split("?")[0].strip()
        return playlist_URI
    except:
        return playlist_URI

In [6]:
with open('./data/spotify_playlists/playlist_links.txt') as links:
    for line in links:
        playlists_uris.append(playlists(line))

print(playlists_uris)

['1G8IpkZKobrIlXcVPoSIuf', '3ONtdDxw2cKFg0hfEdwSCa', '37i9dQZF1DXcBWIGoYBM5M', '31ymdYCITDnZRtkKzP3Itp', '37i9dQZF1DX5hHfOi73rY3', '37i9dQZF1DXa71eg5j9dKZ', '37i9dQZF1DWSvKsRPPnv5o', '37i9dQZF1DXcDoDDetPsEg']


In [7]:
song_info = []
tenk_songs = playlists_uris[:2]

In [8]:

import time

In [13]:
x = 0    
for song in playlists_uris[1:]:
    for track in sp.playlist_tracks(song)["items"]:
            try:
            #URI
                track_uri = track["track"]["uri"]
                
                #Track name
                track_name = track["track"]["name"]
                
                #Main Artist
                artist_uri = track["track"]["artists"][0]["uri"]
                artist_info = sp.artist(artist_uri)
                
                #Name, popularity, genre
                artist_name = track["track"]["artists"][0]["name"]
                artist_pop = artist_info["popularity"]
                artist_genres = artist_info["genres"]
                
                #Album
                album = track["track"]["album"]["name"]
                
                #Popularity of the track
                track_pop = track["track"]["popularity"]

                track_features = sp.audio_features(track_uri)[0]

                track_features['song_name'] = track_name
                track_features['artist_name'] = artist_name
                track_features['artist_genres'] = artist_info["genres"]
                track_features['artist_pop'] = artist_info["popularity"]
                track_features['song_popularity'] = track_pop
                
                song_info.append(track_features)
                
            

            except:
                next
    print(len(song_info))
    time.sleep(30)

print('done')

In [10]:
#load from previously saved extracted song data
small_music_data = pd.read_csv('./data/other_song_data')
tenK_df = pd.read_csv('./data/10k_data')

In [14]:
small_music_data.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop
0,0,0.623,0.763,9,-4.412,1,0.0719,0.19900,0.000007,0.1260,...,6iCJCZqDJjmBxt07Oid6FI,spotify:track:6iCJCZqDJjmBxt07Oid6FI,https://api.spotify.com/v1/tracks/6iCJCZqDJjmB...,https://api.spotify.com/v1/audio-analysis/6iCJ...,226610,4,Buttercup,Hippo Campus,"['indie pop', 'indie poptimism', 'indie rock',...",62
1,1,0.641,0.559,7,-11.132,0,0.0355,0.40400,0.004020,0.0937,...,0Oqc0kKFsQ6MhFOLBNZIGX,spotify:track:0Oqc0kKFsQ6MhFOLBNZIGX,https://api.spotify.com/v1/tracks/0Oqc0kKFsQ6M...,https://api.spotify.com/v1/audio-analysis/0Oqc...,202193,4,Doin' Time,Lana Del Rey,"['art pop', 'pop']",90
2,2,0.647,0.398,4,-8.630,1,0.0283,0.10800,0.107000,0.1480,...,0mO6oS60RST2sWmN2FKknP,spotify:track:0mO6oS60RST2sWmN2FKknP,https://api.spotify.com/v1/tracks/0mO6oS60RST2...,https://api.spotify.com/v1/audio-analysis/0mO6...,432560,4,One More Hour,Tame Impala,"['australian psych', 'neo-psychedelic']",80
3,3,0.662,0.681,11,-8.152,0,0.0315,0.03210,0.139000,0.5080,...,4XfI7M5gLwEY84gYiH4RDD,spotify:track:4XfI7M5gLwEY84gYiH4RDD,https://api.spotify.com/v1/tracks/4XfI7M5gLwEY...,https://api.spotify.com/v1/audio-analysis/4XfI...,186307,4,Devil's Advocate,The Neighbourhood,"['modern alternative rock', 'modern rock', 'po...",85
4,4,0.726,0.815,11,-5.886,1,0.0313,0.00799,0.081000,0.1120,...,3lIxtCaROdRDuTnNBDm3n2,spotify:track:3lIxtCaROdRDuTnNBDm3n2,https://api.spotify.com/v1/tracks/3lIxtCaROdRD...,https://api.spotify.com/v1/audio-analysis/3lIx...,209560,4,She's My Collar (feat. Kali Uchis),Gorillaz,['alternative hip hop'],80


In [11]:
import openai

In [12]:
openai.api_key = os.getenv('OPENAI_SECRET')

In [22]:
small_prompts = []
tenk_prompts = []

In [23]:
for i, r in small_music_data.iterrows():
    small_prompts.append(f"only give the genre of the song {r['song_name']} by {r['artist_name']}")
for i, r in tenK_df.iterrows():
    tenk_prompts.append(f"only give the genre of the song {r['song_name']} by {r['artist_name']}")

In [35]:
print(len(small_prompts))
print(len(tenk_prompts))

505
9947


In [25]:
model = 'text-davinci-002'

In [34]:
small_outputs = []

In [36]:
x = 0

while len(small_outputs) <= (len(small_prompts)-20):
    try:
        completions = openai.Completion.create(engine=model, prompt = small_prompts[x:20 + x], max_tokens=500, temperature=0.5)
        for choice in completions.choices:
            small_outputs.append(choice.text)
        x += 20
        time.sleep(60)
    except:
        completions = openai.Completion.create(engine=model, prompt = small_prompts[-20:], max_tokens=500, temperature=0.5)
        for choice in completions.choices:
            small_outputs.append(choice.text)
        break
else:
    print('done')

done


In [37]:
print(len(small_outputs))

500


In [38]:
#loop didnt grab the last 8
completions = openai.Completion.create(engine=model, prompt = small_prompts[500:], max_tokens=500, temperature=0.5)
for choice in completions.choices:
    small_outputs.append(choice.text)

In [39]:
half_tenk_outputs = []
half_tenk_prompts = tenk_prompts[:4800]

In [40]:
x = 0

while len(half_tenk_outputs) <= (len(half_tenk_prompts)-20):
    try:
        completions = openai.Completion.create(engine=model, prompt = half_tenk_prompts[x:20 + x], max_tokens=500, temperature=0.5)
        for choice in completions.choices:
            half_tenk_outputs.append(choice.text)
        x += 20
        time.sleep(60)
    except:
        completions = openai.Completion.create(engine=model, prompt = half_tenk_prompts[-20:], max_tokens=500, temperature=0.5)
        for choice in completions.choices:
            half_tenk_outputs.append(choice.text)
        break
else:
    print('done')

In [44]:
print(half_tenk_outputs[:10])

['\n\nFolk', '\n\ngrunge', '\n\nRock', '\n\nRock', '\n\nRock and Roll', '\n\nR&B', '\n\nThe Ronettes are a pop girl group from the 1960s. Their song Be My Baby is a pop song with elements of rock and roll.', '\n\nR&B; Soul', '\n\nR&B', '\n\nRock']


In [45]:
print(half_tenk_prompts[:10])

['only give the genre of the song Like a Rolling Stone by Bob Dylan', 'only give the genre of the song Smells Like Teen Spirit by Nirvana', 'only give the genre of the song A Day In The Life - Remastered by The Beatles', 'only give the genre of the song Good Vibrations (Mono) by The Beach Boys', 'only give the genre of the song Johnny B Goode by Chuck Berry', 'only give the genre of the song Respect by Aretha Franklin', 'only give the genre of the song Be My Baby by The Ronettes', "only give the genre of the song What's Going On by Marvin Gaye", 'only give the genre of the song I Heard It Through The Grapevine by Marvin Gaye', "only give the genre of the song (I Can't Get No) Satisfaction - Mono Version / Remastered 2002 by The Rolling Stones"]


In [46]:
def new_line_strip(value):
    return ''.join(value.splitlines())

In [55]:
song_genre_small = []

In [56]:

for item in small_outputs:
    item = new_line_strip(item).lower()
    if 'is' in item:
        split = item.split('is')
        item = split[-1]
        song_genre_small.append(item)
    else:
        song_genre_small.append(item)
    temp_string_list = []

In [59]:
tenk_song_info = []

In [60]:

for item in half_tenk_outputs:
    item = new_line_strip(item).lower()
    if 'is' in item:
        split = item.split('is')
        item = split[-1]
        tenk_song_info.append(item)
    else:
        tenk_song_info.append(item)

In [80]:
small_music_data['song_genre'] = song_genre_small

In [81]:
small_music_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_genres,artist_pop,song_genre
0,0.623,0.763,9,-4.412,1,0.0719,0.19900,0.000007,0.1260,0.199,...,spotify:track:6iCJCZqDJjmBxt07Oid6FI,https://api.spotify.com/v1/tracks/6iCJCZqDJjmB...,https://api.spotify.com/v1/audio-analysis/6iCJ...,226610,4,Buttercup,Hippo Campus,"['indie pop', 'indie poptimism', 'indie rock',...",62,indie rock
1,0.641,0.559,7,-11.132,0,0.0355,0.40400,0.004020,0.0937,0.523,...,spotify:track:0Oqc0kKFsQ6MhFOLBNZIGX,https://api.spotify.com/v1/tracks/0Oqc0kKFsQ6M...,https://api.spotify.com/v1/audio-analysis/0Oqc...,202193,4,Doin' Time,Lana Del Rey,"['art pop', 'pop']",90,indie pop
2,0.647,0.398,4,-8.630,1,0.0283,0.10800,0.107000,0.1480,0.320,...,spotify:track:0mO6oS60RST2sWmN2FKknP,https://api.spotify.com/v1/tracks/0mO6oS60RST2...,https://api.spotify.com/v1/audio-analysis/0mO6...,432560,4,One More Hour,Tame Impala,"['australian psych', 'neo-psychedelic']",80,indie rock
3,0.662,0.681,11,-8.152,0,0.0315,0.03210,0.139000,0.5080,0.864,...,spotify:track:4XfI7M5gLwEY84gYiH4RDD,https://api.spotify.com/v1/tracks/4XfI7M5gLwEY...,https://api.spotify.com/v1/audio-analysis/4XfI...,186307,4,Devil's Advocate,The Neighbourhood,"['modern alternative rock', 'modern rock', 'po...",85,indie rock
4,0.726,0.815,11,-5.886,1,0.0313,0.00799,0.081000,0.1120,0.462,...,spotify:track:3lIxtCaROdRDuTnNBDm3n2,https://api.spotify.com/v1/tracks/3lIxtCaROdRD...,https://api.spotify.com/v1/audio-analysis/3lIx...,209560,4,She's My Collar (feat. Kali Uchis),Gorillaz,['alternative hip hop'],80,r&b


In [87]:
print(len(tenk_song_info))

1920


In [88]:
tenK_df = tenK_df.iloc[0:1920]

In [89]:
tenK_df['song_genre'] = tenk_song_info

In [70]:
tenK_df.drop(columns='Unnamed: 0', inplace=True)
small_music_data.drop(columns='Unnamed: 0', inplace=True)

In [90]:
combined = pd.concat([tenK_df,small_music_data])
combined.reset_index(inplace=True)

In [91]:
combined.drop(columns='index', inplace=True)

In [92]:
combined.to_csv('./data/updated_music.csv')